In [2]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
from sklearn.pipeline import make_pipeline

In [4]:
import mlflow


## after configuring aws instance connection run this : 

mlflow.set_tracking_uri("http://35.77.101.3:5000")
mlflow.set_experiment("green-taxi-duration")



<Experiment: artifact_location='s3://mlflow-artifacts-dir-2/1', creation_time=1674370309730, experiment_id='1', last_update_time=1674370309730, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [5]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [6]:
df_train = read_dataframe('/home/lambu/Desktop/Project2022_2023/MLOps_Zoomcamp_1/input_raw_data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/lambu/Desktop/Project2022_2023/MLOps_Zoomcamp_1/input_raw_data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

After reading the data run the below cell and view in server to see the changes i.e. artifacts will store in s3 buckets

In [7]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=50, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)
        
    mlflow.sklearn.log_model(pipeline, artifact_path="model")

    #mlflow.log_artifact("preprocessor.b")
    #mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
    #mlflow.log_artifacts('dict_vectorizer.bin',artifact_path="model")



{'max_depth': 20, 'n_estimators': 50, 'min_samples_leaf': 10, 'random_state': 0} 6.7557528374758675


2023/01/22 15:00:41 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp8obu4jrp/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.2.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
/home/lambu/anaconda3/envs/deployment_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [8]:
from mlflow.tracking import MlflowClient

In [9]:
MLFLOW_TRACKING_URI = 'http://35.77.101.3:5000'

RUN_ID = '49067d17e9994a0dadac7f5f7c97942f'   ## replace with latest run id

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


In [10]:
path = client.download_artifacts(run_id=RUN_ID, path='')  # path='dict_vectorizer.bin'

/tmp/ipykernel_11992/2448725023.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id=RUN_ID)  # path='dict_vectorizer.bin'


TypeError: download_artifacts() missing 1 required positional argument: 'path'

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [ ]:
dv